In [1]:
import gzip
import json
import csv
import pandas as pd
import re

from pandas import json_normalize

import my_functions
%load_ext autoreload
%autoreload 2

Se realiza la carga del archivo original a un dataframe, 'australian_user_reviews.json', extraído previamente dado que venía comprimido.

In [2]:
with open("Datasets/Datasets originales/australian_user_reviews.json", 'r', encoding='utf-8') as data:
    data = data.readlines()
    
data = [eval(line) for line in data]

df_reviews = pd.DataFrame(data)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [3]:
df_reviews.to_csv('Datasets/Datasets ETL/user_reviews_raw.csv', index=False, encoding='utf-8')

Se comienza a realizar una exploración inicial.

In [4]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


### 'user_id'

Se explora el campo 'user_id', entendiendo que el mismo debería ser el campo principal dado que este dataset refiere a reseñas de usuarios.

In [5]:
valores_unicos_user_id = df_reviews['user_id'].nunique()
print(f"Cantidad total de registros': {len(df_reviews)}")
print(f"Número de valores únicos en 'user_id': {valores_unicos_user_id}")

Cantidad total de registros': 25799
Número de valores únicos en 'user_id': 25485


Se observa que hay una pequeña parte de registros con 'user_id' duplicados.

In [6]:
# Identificar registros duplicados en 'id_user'
duplicados = df_reviews[df_reviews.duplicated(subset='user_id', keep=False)].sort_values('user_id')

# Guardar columnas 'user_id' y 'reviews' ordenadas por 'user_id'
resultados = duplicados[['user_id', 'reviews']].sort_values('user_id')

# Configurar la opción para mostrar más filas
# pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows')
# resultados

resultados.head()



,user_id,reviews
12888,05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
5250,05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
3133,111222333444555666888,"[{'funny': '', 'posted': 'Posted December 22, ..."
3134,111222333444555666888,"[{'funny': '', 'posted': 'Posted December 22, ..."
4139,29123,"[{'funny': '', 'posted': 'Posted March 26.', '..."


Habiendo recorrido los 314 registros con 'user_id' duplicado, se determina que son registros con todos los campos duplicados, por lo cual se procede a eliminar el segundo de cada uno.

In [7]:
# Eliminar segundo registro duplicado
df_reviews = df_reviews.drop_duplicates(subset='user_id', keep='first')

Se verifican las modificaciones, pudiendo observar que el total de registros cuentan con un 'user_id' único.

In [8]:
my_functions.duplicates(df_reviews, 'user_id')

valores_unicos_user_id = df_reviews['user_id'].nunique()
print(f"Cantidad total de registros': {len(df_reviews)}")
print(f"Número de valores únicos en 'user_id': {valores_unicos_user_id}")

Cantidad de valores duplicados para user_id : 0
Valores duplicados: 
 []
Cantidad total de registros': 25485
Número de valores únicos en 'user_id': 25485


### Campo 'reviews'

Cada valor de este campo cuenta con una lista, dentro de la cual se encuentra un diccionario para cada reseña realizada por el usuario dado.

In [9]:
df_reviews['reviews'][1]

[{'funny': '',
  'posted': 'Posted June 24, 2014.',
  'last_edited': '',
  'item_id': '251610',
  'helpful': '15 of 20 people (75%) found this review helpful',
  'recommend': True,
  'review': 'I know what you think when you see this title "Barbie Dreamhouse Party" but do not be intimidated by it\'s title, this is easily one of my GOTYs. You don\'t get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can\'t 360 noscope your friends, but what you can do is show them up with your bad ♥♥♥ dance moves and put them to shame as you show them what true fashion and color combinations are.I know this game says for kids but, this is easily for any age range and any age will have a blast playing this.8/8'},
 {'funny': '',
  'posted': 'Posted September 8, 2013.',
  'last_edited': '',
  'item_id': '227300',
  'helpful': '0 of 1 people (0%) found this review helpful',
  'recommend': True,
  'review': "For a simple (it's actually not all th

Se procederá a desanidar este campo de forma tal que contaremos con una fila por cada reseña que haya realizado un usuario. De esta forma, ya no contaremos con valores únicos para el 'user_id'.

In [10]:
# Desanidar la columna 'items'
df_reviews = pd.json_normalize(data, record_path=['reviews'], meta=['user_id','user_url'] )
df_reviews.head(3)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...


Se verifica que se han generado nuevos registros de reseñas para cada usuario. Como se puede observar, ahora se cuenta con más de un registro por usuario, en caso que haya realizado más de una reseña.

Se continua con la exploración para definir si hay campos o registros que no resulten de utilidad y proceder a eliminarlos.

In [11]:
my_functions.describe_df(df_reviews)

Cantidad Registros:  59305
Cantidad Campos:  9


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,funny,object,0,0.0,59305,100.0,186,0.31
1,posted,object,0,0.0,59305,100.0,1906,3.21
2,last_edited,object,0,0.0,59305,100.0,1015,1.71
3,item_id,object,0,0.0,59305,100.0,3682,6.21
4,helpful,object,0,0.0,59305,100.0,1344,2.27
5,recommend,bool,0,0.0,59305,100.0,2,0.00
6,review,object,0,0.0,59305,100.0,55313,93.27
7,user_id,object,0,0.0,59305,100.0,25458,42.93
8,user_url,object,0,0.0,59305,100.0,25458,42.93


### Eliminacion de campos no relevantes

Se decide eliminar, en primer lugar, registros con 'user_id' nulo, que representan un valor poco significativo del dataset.

También se eliminan los campos 'user_url', 'funny', 'last_edited', 'helpful', considerando que los mismos no aportan datos relevantes ni son necesarios a los efectos de los objetivos.

In [12]:
# Quitar registros con user_id nulo
df_reviews = df_reviews.dropna(subset=['user_id'])

# Remover columnas del DataFrame df_reviews
columnas_a_eliminar_reviews = ['user_url', 'funny', 'last_edited', 'helpful']
df_reviews = df_reviews.drop(columns=columnas_a_eliminar_reviews, errors='ignore')

### Registros duplicados

Se investiga si hay filas duplicadas.

In [13]:
# Configurar la visualización
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

# Suponiendo que df_reviews es tu DataFrame
duplicados = df_reviews[df_reviews.duplicated(keep=False)].sort_values(by=['user_id', 'item_id'])

# Mostrar todas las filas duplicadas
duplicados



,posted,item_id,recommend,review,user_id
13411,"Posted January 5, 2015.",277430,True,this is the best third person game ever that i...,05041129
31985,"Posted January 5, 2015.",277430,True,this is the best third person game ever that i...,05041129
13412,"Posted May 23, 2015.",440,False,this will be the number one game if it have m...,05041129
31986,"Posted May 23, 2015.",440,False,this will be the number one game if it have m...,05041129
13410,"Posted May 18, 2015.",730,True,This game to me it is so good that it is bette...,05041129
...,...,...,...,...,...
43716,"Posted October 31, 2014.",250320,True,"from the creaters of the walking dead, i prese...",yolofaceguy
6752,"Posted September 2, 2014.",261030,True,"this game is awesome,this game is ♥♥♥♥ed up an...",yolofaceguy
43717,"Posted September 2, 2014.",261030,True,"this game is awesome,this game is ♥♥♥♥ed up an...",yolofaceguy
14835,"Posted November 30, 2013.",219640,True,SAVE THE KING!This is one of my favourite LAN ...,zeroblade


In [14]:
# Contar filas duplicadas
total_duplicadas = df_reviews.duplicated().sum()

# Mostrar el total de filas duplicadas
print(f"Total de filas duplicadas: {total_duplicadas}")

Total de filas duplicadas: 874


Se identifican 874 filas duplicadas.
Se procede a eliminarlas quedando solo el primer registro de cada una.

In [15]:
# Suponiendo que df_reviews es tu DataFrame
df_reviews = df_reviews.drop_duplicates()

# Contar filas duplicadas
total_duplicadas = df_reviews.duplicated().sum()

# Mostrar el total de filas duplicadas
print(f"Total de filas duplicadas: {total_duplicadas}")


Total de filas duplicadas: 0


### Campo 'item_id' ('game_id' en adelante)

Se identifica que el campo 'item_id' se corresponde con el campo 'game_id' que se trató en '1.1 ETL games', por lo cual se modifica su nombre para unificarlo como campo común a los datasets.

In [16]:
# Cambiar el nombre de la columna 'item_id' a 'game_id'
df_reviews.rename(columns={'item_id': 'game_id'}, inplace=True)

In [17]:
my_functions.describe_df(df_reviews)

Cantidad Registros:  58431
Cantidad Campos:  5


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,posted,object,0,0.0,58431,100.0,1906,3.26
1,game_id,object,0,0.0,58431,100.0,3682,6.30
2,recommend,bool,0,0.0,58431,100.0,2,0.00
3,review,object,0,0.0,58431,100.0,55313,94.66
4,user_id,object,0,0.0,58431,100.0,25458,43.57


Se puede observar que hay reseñas de 3682 juegos.

### Campo 'posted'

Se pueden observar valores con texto referido a la fecha de posteo.

Se trabaja sobre el campo 'posted' para limpiarlo y extraer un valor númerico para el año de posteo de la reseña en un nuevo campo ('posted_year').

In [18]:
df_reviews['posted']

0         Posted November 5, 2011.
1            Posted July 15, 2011.
2           Posted April 21, 2011.
3            Posted June 24, 2014.
4        Posted September 8, 2013.
                   ...            
59300              Posted July 10.
59301               Posted July 8.
59302               Posted July 3.
59303              Posted July 20.
59304               Posted July 2.
Name: posted, Length: 58431, dtype: object

In [19]:
# Función para extraer el año o devolver 0 si no se puede extraer
def extract_year_or_zero(posted_text):
    try:
        year = int(re.search(r'(\d{4})', posted_text).group(1))
        return year
    except (AttributeError, TypeError):
        return 0

# Aplicar la función a la columna 'posted' y crear la columna 'posted_year'
df_reviews['posted_year'] = df_reviews['posted'].apply(extract_year_or_zero)

In [20]:
df_reviews['posted_year']

0        2011
1        2011
2        2011
3        2014
4        2013
         ... 
59300       0
59301       0
59302       0
59303       0
59304       0
Name: posted_year, Length: 58431, dtype: int64

Se observa que existen reseñas para los años 2010 a 2015 inclusive, además de reseñan para las cuales no se cuenta con el año correspondiente, identificadas con un 0.

In [21]:
df_reviews['posted_year'].unique()

array([2011, 2014, 2013,    0, 2015, 2012, 2010], dtype=int64)

De un total de 58431, no se cuenta con el año de publicación para 9933.

In [22]:
# Suponiendo que df_reviews es tu DataFrame
contador_cero = df_reviews[df_reviews['posted_year'] == 0].shape[0]

# Mostrar la cantidad de filas donde 'posted_year' es igual a 0
print(f"Cantidad de filas con 'posted_year' igual a 0: {contador_cero}")


Cantidad de filas con 'posted_year' igual a 0: 9933


Se exploran estos registros.

In [23]:
# Configurar la visualización
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

# Suponiendo que df_reviews es tu DataFrame
filas_posted_year_0 = df_reviews[df_reviews['posted_year'] == 0]

# Filtrar filas donde 'posted' es duplicado
filas_sin_duplicados_posted = filas_posted_year_0[~filas_posted_year_0['posted'].duplicated()]

# Mostrar el DataFrame resultante
filas_sin_duplicados_posted


,posted,game_id,recommend,review,user_id,posted_year
6,Posted February 3.,248820,True,A suitably punishing roguelike platformer. Wi...,evcentric,0
27,Posted May 20.,730,True,ZIKA DO BAILE,76561198079601835,0
28,Posted July 24.,730,True,BEST GAME IN THE BLOODY WORLD,MeaTCompany,0
31,Posted June 16.,252950,True,love it,76561198156664158,0
32,Posted June 11.,440,True,mt bom,76561198077246154,0
...,...,...,...,...,...,...
5187,Posted March 17.,301520,False,Classic example of poorly managed feature cree...,Sam_Slade,0
5785,Posted May 22.,213670,False,Yes it is a Really fun game but it becomes poi...,superduperjack,0
6207,Posted April 6.,262410,True,"World of Guns: Gun Disassembly is a good game,...",63Hayden,0
6290,Posted August 11.,570,True,เกมนี้เป็นเกมที่สนุกมากจนต้องเติมตังซื้อของเลย...,chrisfoong,0


Se verifica que en estos casos efectivamente no se cuenta con información para extraer el año de publicacipon.

Se descarta el campo 'posted' dado que ya no resulta de utilidad.

In [24]:
df_reviews = df_reviews.drop(columns='posted')

### Campo 'recommend'

Este campo es de valor booleano (True / False), indicando si en dicha reseña se recomienda o no el juego en cuestión.

No requiere transformaciones.

In [25]:
df_reviews['recommend'].unique()

array([ True, False])

### Campo 'review'

En este campo nos encontramos con el texto del la reseña.

In [26]:
# Ejemplo
df_reviews['review'][1]

"It's unique and worth a playthrough."

Se exploran duplicados.

In [27]:
# Configurar la visualización
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

my_functions.duplicates(df_reviews, 'review')

Cantidad de valores duplicados para review : 767
Valores duplicados: 
 ['' ' ' '!' '!!!!!'
 '"Cheap, fun, basic, and the developers hate DLC as much as we do. If it\'s on sale it usually goes under a dollar. WORTH IT! Also good multi-player!'
 '#1' '(Y)' '(y)'
 "*Bane's Voice* I'm a simple man. I see microtransactions i downvote."
 ',' '.' '..' '...' '....'
 '........_......./░\\....../..N.\\...../░O░\\.....||||||||||.....|░P░|.....|░A░|.....|░I.░|.....|░D░|...BOB IS BUILDING MISSILES TO FIGHT.....||||||||||...AGAINST PAID MODS!.....|░M░| .....|░O░| .....|░D░|._-"|░S░|"-_|=========|...\\☻/︻╦╤─ ..../#####\\..........▌...."""""""""........../ \\... JOIN HERE -> http://steamcommunity.com/groups/removepaidmods/'
 '0/10' '1' '10/10' '10/10 IGN' '10/10 Would buy again'
 '10/10 Would play again.' '10/10 best game' '10/10 best game ever'
 '10/10 great game' '10/10 ign' '10/10 would bang'
 '10/10 would bang again' '10/10 would bang.' '10/10 would fly again'
 '10/10 would play again' '10/10 woul

,game_id,recommend,review,user_id,posted_year
56476,224540,True,,PLZACCEPTTHEURL,2014
28304,440,True,,KamuKambing,2013
45386,730,True,,viihdavanzo,2015
45385,238960,True,,viihdavanzo,2015
20478,378041,True,,76561197971285616,0
...,...,...,...,...,...
56870,253710,False,♥♥♥♥♥♥,420BlazeItMLG,0
9447,730,True,♥♥♥♥♥♥,brdzi,2015
24052,4000,True,♥♥♥♥♥♥,1850172512752815,0
52872,440,True,꿀잼,76561198072942552,2015


Se encuentran 767 valores de 'review' duplicados entre 3885 registros.

Luego de visualizar estos registros se determina que no representan una inconsistencia en la información.

Algunos de los valores son vacío, puntaje numérico, una palabra descriptiva y otros ejemplos. En algunos casos reseñas mas elaboradas repetidas por un mismo usuario, pero para distintos juegos.

Se deja esta información como se encuentra.

### Campo 'sentiment'

Uno de los objetivos de este proyecto es trabajar con un análisis de sentimientos de las reseñas, para lo cual se tomarán los valores de 'review' para aplicar el algoritmo correspondiente.

Se genera un nuevo campo, 'sentiment' que contendrá como valores 0, 1 o 2, representando un sentimiento negativo, neutral o positivo respectivamente.

In [28]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

# Instanciar el analizador de sentimientos
analyzer = SentimentIntensityAnalyzer()

# Función para obtener la etiqueta de sentimiento

def get_sentiment_label(compound_score):
    if compound_score >= 0.01: 
        return 2  # Positivo
    elif compound_score <= -0.04:  
        return 0  # Negativo
    else:
        return 1  # Neutral



# Aplicar el análisis de sentimiento y crear el DataFrame auxiliar df_sentiment
sentiments = []
reviews = df_reviews['review'].tolist()

for review in reviews:
    sentiment_scores = analyzer.polarity_scores(review)
    sentiment_label = get_sentiment_label(sentiment_scores['compound'])
    sentiments.append(sentiment_label)

# Agregar la columna 'sentiment' a df_reviews
df_reviews['sentiment'] = sentiments

df_reviews.head(1)

,game_id,recommend,review,user_id,posted_year,sentiment
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011,2


Se exploran algunos registros generados para cada sentimiento.

In [29]:
# Configurar Pandas para mostrar el contenido completo de las columnas
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [30]:
# Mostrar 5 registros para cada valor de sentiment (0, 1, 2)
for sentiment_value in [0]:
    df_subset = df_reviews[df_reviews['sentiment'] == sentiment_value].head(5)
    print(f"\nSentiment {sentiment_value} (negativo):\n")

df_subset[['review', 'sentiment']]



Sentiment 0 (negativo):



,review,sentiment
6,A suitably punishing roguelike platformer. Winning feels good. Progressive unlocks mean a good slog ending in failure doesn't feel like a waste.,0
23,"It reminds me of that TV Show called ""The Walking Dead"".",0
29,"Killed the Emperor, nobody cared and got away with it. Accidentally killed a chicken and everybody decided to gang up on me. 10/10",0
43,"Compared to Left 4 Dead 2, this game REALLY gives you the creeps and the adrenaline rush to kill the Patrioch. +1 for this gameAlthough it does need some major improvements to how they put in the designs (Not the zombies, the HUDs).Other than that, GREAT game!Thrilling!",0
56,"If you're looking for a no frills, Arcade/Simulator Style Dog fighting game you have come to the right place.",0


In [31]:
for sentiment_value in [1]:
    df_subset = df_reviews[df_reviews['sentiment'] == sentiment_value].head(15)
    print(f"\nSentiment {sentiment_value} (neutral):\n")

df_subset[['review', 'sentiment']]



Sentiment 1 (neutral):



,review,sentiment
9,"Random drops and random quests, with stat points. Animation style reminiscent of the era before the Voodoo card.",1
18,Git gud,1
22,This game is Marvellous.,1
27,ZIKA DO BAILE,1
30,10/10 would eat your money for hats and keys,1
32,mt bom,1
33,É muito bom,1
34,es la rajaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,1
35,MAGICKA *SNORT* *SNORT*,1
47,What a Awsome game!,1


In [32]:
for sentiment_value in [2]:
    df_subset = df_reviews[df_reviews['sentiment'] == sentiment_value].head(15)
    print(f"\nSentiment {sentiment_value} (positivo):\n")

df_subset[['review', 'sentiment']]



Sentiment 2 (positivo):



,review,sentiment
0,"Simple yet with great replayability. In my opinion does ""zombie"" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth ""zombie"" splattering fun for the whole family. Amazed this sort of FPS is so rare.",2
1,It's unique and worth a playthrough.,2
2,Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!,2
3,"I know what you think when you see this title ""Barbie Dreamhouse Party"" but do not be intimidated by it's title, this is easily one of my GOTYs. You don't get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can't 360 noscope your friends, but what you can do is show them up with your bad ♥♥♥ dance moves and put them to shame as you show them what true fashion and color combinations are.I know this game says for kids but, this is easily for any age range and any age will have a blast playing this.8/8",2
4,"For a simple (it's actually not all that simple but it can be!) truck driving Simulator, it is quite a fun and relaxing game. Playing on simple (or easy?) its just the basic WASD keys for driving but (if you want) the game can be much harder and realistic with having to manually change gears, much harder turning, etc. And reversing in this game is a ♥♥♥♥♥, as I imagine it would be with an actual truck. Luckily, you don't have to reverse park it but you get extra points if you do cause it is bloody hard. But this is suprisingly a nice truck driving game and I had a bit of fun with it.",2
5,Very fun little game to play when your bored or as a time passer. Very gud. Do Recommend. pls buy,2
7,"""Run for fun? What the hell kind of fun is that?""",2
8,"Elegant integration of gameplay, story, world development and aesthetic.",2
10,Fun balance of tactics and strategy. Potential for very rewarding battles on smaller maps. Can become a bit of a grind on larger maps (>200 stars).,2
11,"Fun world builder, with plenty of option of how you want challenge served to you. Gnome pathing sometimes frustrating if you expand very very quickly.",2


En una primer observación, se considera que el análisis de sentimiento de las reseñas no resulta muy preciso.

Sin embargo, tras haber probado distintas alternativas, se determina avanzar, explorar los valores generados para 'sentiment' en el EDA y evaluar posibilidades de mejora a futuro.

El campo 'review' se elimina.

In [33]:
# Suponiendo que df_reviews es tu DataFrame
df_reviews = df_reviews.drop(columns=['review'])

Se explora nuevamente el dataset siendo esta la versión que se utilizará para continuar en el EDA.

In [34]:
my_functions.describe_df(df_reviews)

Cantidad Registros:  58431
Cantidad Campos:  5


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,game_id,object,0,0.0,58431,100.0,3682,6.30
1,recommend,bool,0,0.0,58431,100.0,2,0.00
2,user_id,object,0,0.0,58431,100.0,25458,43.57
3,posted_year,int64,0,0.0,58431,100.0,7,0.01
4,sentiment,int64,0,0.0,58431,100.0,3,0.01


### 'user_reviews_v1'

Se guarda el archivo con las modificaciones realizadas.

In [35]:
df_reviews.to_csv('Datasets/Datasets ETL/user_reviews_v1.csv', index=False, encoding='utf-8')

Se guarda el nuevo dataset como 'user_reviews_v1.csv', para utilizarlo en próximos pasos del proyecto.